In [34]:
import os
import pandas as pd
import numpy as np
from nltk import wordpunct_tokenize
import torch

#import preprocessing file
import pp

In [35]:
asp = {
"NAME" : 1,
"CITATION" : 2,
"COUNSEL" : 3,
"JUDGE" : 4,
"FACTS" : 5,
"RLC" : 6,
"REASONING" : 7,
"ARG" : 8,
"STATUTE" : 9,
"PRECEDENT" : 10,
"RPC" : 11,
"ISSUE" : 12
}

In [36]:
x = []
y = []
no_of_files = 10

In [37]:
csvFolderPath = '../labelled_data/'
for filename in os.listdir(csvFolderPath):
    print(filename[:-4] + '.txt')

6.txt
7.txt
5.txt
4.txt
1.txt
3.txt
2.txt
16.txt
15.txt
14.txt
10.txt
13.txt
12.txt
9.txt
8.txt


In [38]:
csvFolderPath = '../labelled_data/'
for filename in os.listdir(csvFolderPath):
    print(filename)
    annpath = os.path.join(csvFolderPath, filename)
    df = pd.read_csv(annpath)
    df = df.dropna(subset='Sentence',how='all')

    #now each row in df has some value and are not empty
    text = []
    for index,row in df.iterrows():
        text.append(row['Sentence'])
    x.append(pp.preprocess(text))
    temp = []
    for index, row in df.iterrows():
        if pd.isna(row['Labels']):
            dum = [0]
        else:
            dum = []
            for word in wordpunct_tokenize(row['Labels']):
                if word in asp:
                    dum.append(asp[word])
            if len(dum)==0:
                dum.append(0)
        temp.append(dum)
    y.append(temp)

6.csv
7.csv
5.csv
4.csv
1.csv
3.csv
2.csv
16.csv
15.csv
14.csv
10.csv
13.csv
12.csv
9.csv
8.csv


In [39]:
for i in range(0,10):
    if len(x[i])!=len(y[i]):
        print(i)

In [27]:
import pickle
# Save data to a pickle file
with open('../intermediate/x.pickle', 'wb') as file:
    pickle.dump(x, file)

with open('../intermediate/y.pickle', 'wb') as file:
    pickle.dump(y, file)